In [16]:
import os
import json
import numpy as np
import pandas as pd
import rasterio
from rasterio.mask import mask
import geopandas as gpd

In [29]:
def normalize_pixel_fractions(fractions, modified_indices, percentage_changes):
    f = fractions.copy()
    for i, idx in enumerate(modified_indices):
        f[idx] *= (1 + percentage_changes[i])
    f = np.clip(f, 0, 1)
    mod_sum = np.sum(f[modified_indices])
    mod_sum = min(mod_sum, 1.0)
    unmodified_indices = [i for i in range(len(f)) if i not in modified_indices]
    unmod_sum = np.sum(f[unmodified_indices])
    if unmod_sum > 0:
        scale = (1.0 - mod_sum) / unmod_sum
        for i in unmodified_indices:
            f[i] *= scale
    else:
        f[modified_indices[0]] += (1.0 - mod_sum)
    correction = 1.0 - np.sum(f)
    if abs(correction) > 1e-6:
        fallback = modified_indices[0] if modified_indices else 0
        f[fallback] += correction
    return f

def layer_alterator(raster_folder, vector_path, operation_rule_path, output_folder=None, value_range=(0, 1)):
    with open(operation_rule_path, 'r') as f:
        operation_rules = json.load(f)

    mask_gdf = gpd.read_file(vector_path)
    raster_files = []
    for root, dirs, files in os.walk(raster_folder):
        for file in files:
            if file.lower().endswith(('.tif', '.tiff')):
                raster_files.append(os.path.join(root, file))

    lc_layers = [f for f in operation_rules if f.startswith("F_")]
    lc_modes_raw = [operation_rules[f] for f in lc_layers]
    lc_modes = set(m for m in lc_modes_raw if m is not None)
    lc_none_count = sum(1 for m in lc_modes_raw if m is None)

    if len(lc_modes) > 1:
        raise ValueError("❌ Inconsistent operation rules for land cover fractions. You are mixing 'replace' and 'pct'. Please choose only one and update both the vector mask and the JSON rule file accordingly.")
    elif len(lc_modes) == 1:
        lc_mode = list(lc_modes)[0]
        if lc_none_count > 0:
            raise ValueError(f"⚠️ Some land cover fraction layers are set to None while others use '{lc_mode}'. Please either set all to '{lc_mode}', or modify the code to handle partial layers. Optionally, you may update your JSON to convert the None entries to '{lc_mode}'.")
    else:
        print("[INFO] No active operation mode found for LC fractions. Skipping LC validation.")
        lc_mode = None

    if lc_mode == "pct":
        lc_raster_paths = [os.path.join(raster_folder, f) for f in lc_layers]
        lc_rasters = []
        meta = None
        for f in lc_raster_paths:
            with rasterio.open(f) as src:
                if meta is None:
                    meta = src.meta
                lc_rasters.append(src.read(1))
        lc_rasters = np.stack(lc_rasters, axis=0)
        height, width = lc_rasters.shape[1:]
        affine = meta["transform"]

        for _, row in mask_gdf.iterrows():
            geom = row.geometry
            mask_array = rasterio.features.geometry_mask([geom], transform=affine, invert=True, out_shape=(height, width))
            pct_changes = []
            mod_indices = []
            for i, fname in enumerate(lc_layers):
                attr_name = fname.replace(".tif", "")
                val = row.get(attr_name.upper())
                if pd.notna(val):
                    try:
                        pct_changes.append(float(val))
                        mod_indices.append(i)
                    except ValueError:
                        print(f"[SKIP] Invalid percentage for {attr_name.upper()} in row {row.name}: {val}")

            rows, cols = np.where(mask_array)
            for y, x in zip(rows, cols):
                pixel = lc_rasters[:, y, x]
                lc_rasters[:, y, x] = normalize_pixel_fractions(pixel, mod_indices, pct_changes)

        for i, fname in enumerate(lc_layers):
            output_path = os.path.join(output_folder, fname) if output_folder else f"modified_{fname}"
            with rasterio.open(
                output_path, "w", **meta
            ) as dst:
                dst.write(lc_rasters[i], 1)
            print(f"[DONE] Modified {fname} with 'pct' → {output_path}")
        return

    for raster_path in raster_files:
        raster_filename = os.path.basename(raster_path)
        raster_name = os.path.splitext(raster_filename)[0]
        attribute_field = raster_name.upper()
        operation_rule = operation_rules.get(raster_filename)
        if operation_rule is None:
            print(f"[SKIP] No operation rule for '{raster_filename}'")
            continue
        if attribute_field not in mask_gdf.columns:
            print(f"[SKIP] Attribute '{attribute_field}' not found in vector data.")
            continue

        with rasterio.open(raster_path) as src:
            raster_data = src.read(1)
            transform = src.transform
            crs = src.crs
            dtype = src.dtypes[0]
            nodata = src.nodata

            for _, row in mask_gdf.iterrows():
                geometry = [row["geometry"]]
                masked_data, _ = mask(src, geometry, crop=False)
                masked_data = masked_data[0]
                polygon_mask = masked_data != nodata if nodata is not None else masked_data != 0

                if operation_rule == "replace" and raster_filename in lc_layers:
                    total = 0.0
                    for lf in lc_layers:
                        val = row.get(lf.replace(".tif", "").upper())
                        if pd.notna(val):
                            total += float(val)
                    if total > 1.0:
                        raise ValueError(f"❌ In polygon {row.name}, sum of LC replacement values exceeds 1: {total:.2f}. Please adjust your vector attributes.")

                if pd.isna(row.get(attribute_field)):
                    continue
                try:
                    value = float(row[attribute_field])
                except (ValueError, TypeError):
                    print(f"[SKIP] Invalid value for '{attribute_field}': {row[attribute_field]}")
                    continue

                if operation_rule == "pct":
                    modified = masked_data * (1 - value / 100.0)
                elif operation_rule == "replace":
                    modified = np.full_like(masked_data, value)
                else:
                    raise ValueError(f"Unknown operation_rule '{operation_rule}' for '{raster_filename}'")

                modified = np.clip(modified, *value_range)
                raster_data[polygon_mask] = modified[polygon_mask]

        if output_folder:
            os.makedirs(output_folder, exist_ok=True)
            output_path = os.path.join(output_folder, raster_filename)
        else:
            output_path = f"modified_{raster_filename}"

        with rasterio.open(
            output_path, "w", driver="GTiff",
            height=raster_data.shape[0], width=raster_data.shape[1],
            count=1, dtype=dtype, crs=crs, transform=transform
        ) as dst:
            dst.write(raster_data, 1)

        print(f"[DONE] Modified {raster_filename} with '{operation_rule}' → {output_path}")

In [30]:
layer_alterator(
    raster_folder="./test_data/lc_fractions",
    vector_path="./test_data/sample_mask.geojson",
    operation_rule_path="./test_data/operation_rules.json",
    output_folder="./output/modified_raster/new_data",
    value_range=(0, 1)
)

[DONE] Modified F_AC.tif with 'pct' → ./output/modified_raster/new_data/F_AC.tif
[DONE] Modified F_S.tif with 'pct' → ./output/modified_raster/new_data/F_S.tif
[DONE] Modified F_M.tif with 'pct' → ./output/modified_raster/new_data/F_M.tif
[DONE] Modified F_BS.tif with 'pct' → ./output/modified_raster/new_data/F_BS.tif
[DONE] Modified F_G.tif with 'pct' → ./output/modified_raster/new_data/F_G.tif
[DONE] Modified F_TV.tif with 'pct' → ./output/modified_raster/new_data/F_TV.tif
[DONE] Modified F_W.tif with 'pct' → ./output/modified_raster/new_data/F_W.tif
